# Building noise channels

In [ ]:
from quantanium import *

In [2]:
p = 0.1
ch = PauliX(p)

print(ch)

PauliX(0.1)


In [3]:
ch.krausoperators()

[1-qubit Operator:
 ├── 0.948683298050514 0
 └── 0 0.948683298050514,
 1-qubit Operator:
 ├── 0 0.316227766016838
 └── 0.316227766016838 0]

In [4]:
ch.ismixedunitary()

True

In [5]:
ch.unitarymatrices()

[[1.0, 0]
 [0, 1.0],
 [0, 1.0]
 [1.0, 0]]

In [6]:
ps = [0.8, 0.1, 0.1]
paulis = ["II", "XX", "YY"]
PauliNoise(ps, paulis)

PauliNoise((0.8, pauli"II"), (0.1, pauli"XX"), (0.1, pauli"YY"))

In [7]:
from symengine import Matrix

ps = [0.9, 0.1]
E1 = GateID().matrix()
E2 = GateZ().matrix()

MixedUnitary(ps, [E1, E2])

MixedUnitary((0.9, "Custom([1.0 0; 0 1.0])"), (0.1, "Custom([1.0 0; 0 -1.0])"))

In [8]:
from symengine import Matrix

E1 = Matrix([[1, 0], [0, (0.9)**0.5]])
E2 = Matrix([[0, (0.1)**0.5], [0, 0]])

Kraus([E1, E2])

Kraus(Operator([[1, 0], [0, 0.948683298050514]]), Operator([[0, 0.316227766016838], [0, 0]]))

In [9]:
# Create a simple Pauli-X channel
p = 0.1
ch = PauliX(p)

# Properties
ch.krausoperators()
ch.ismixedunitary()
ch.unitarymatrices()

# Generic Pauli
ps = [0.8, 0.1, 0.1]
paulis = ["II", "XX", "YY"]
PauliNoise(ps, paulis)

from symengine import Matrix

# Mixed unitary
ps = [0.9, 0.1]
E1 = GateID().matrix()
E2 = GateZ().matrix()

MixedUnitary(ps, [E1, E2])

# Generic Kraus Channel
E1 = Matrix([[1, 0], [0, (0.9)**0.5]])
E2 = Matrix([[0, (0.1)**0.5], [0, 0]])

Kraus([E1, E2])

Kraus(Operator([[1, 0], [0, 0.948683298050514]]), Operator([[0, 0.316227766016838], [0, 0]]))

# Adding noise one by one

In [10]:
# Example Noisy GHZ state preparation
c = Circuit()

# preparation like error
c.push(PauliX(0.1), range(5))

# Noisy gate-H
c.push(GateH(), 1)
c.push(AmplitudeDamping(0.1), 1)

# Noisy gate-CNOT
c.push(GateCX(), 0, range(1,5))
c.push(Depolarizing2(0.1), 0, range(1,5))

# Noisy Measurements
c.push(PauliX(0.1), range(5))
c.push(Measure(), range(5), range(5))

print(c)

5-qubit circuit with 25 instructions:
├── PauliX(0.1) @ q[0]
├── PauliX(0.1) @ q[1]
├── PauliX(0.1) @ q[2]
├── PauliX(0.1) @ q[3]
├── PauliX(0.1) @ q[4]
├── H @ q[1]
├── AmplitudeDamping(0.1) @ q[1]
├── CX @ q[0], q[1]
├── CX @ q[0], q[2]
├── CX @ q[0], q[3]
├── CX @ q[0], q[4]
├── Depolarizing(0.1) @ q[0,1]
├── Depolarizing(0.1) @ q[0,2]
├── Depolarizing(0.1) @ q[0,3]
├── Depolarizing(0.1) @ q[0,4]
├── PauliX(0.1) @ q[0]
├── PauliX(0.1) @ q[1]
├── PauliX(0.1) @ q[2]
├── PauliX(0.1) @ q[3]
├── PauliX(0.1) @ q[4]
├── M @ q[0], c[0]
├── M @ q[1], c[1]
├── M @ q[2], c[2]
├── M @ q[3], c[3]
└── M @ q[4], c[4]


# Adding noise to all gates of the same type

In [11]:
# Example Noisy GHZ state preparation
# starts by resetting all qubits
c = Circuit()
c.push(Reset(), range(5))
c.push(GateH(), 0)
c.push(GateCX(), 0, range(1,5))
c.push(Measure(), range(5), range(5))

# Adding Noise
c.add_noise(Reset(), PauliX(0.1), parallel=True)
c.add_noise(GateH(), AmplitudeDamping(0.1))
c.add_noise(GateCX(), Depolarizing2(0.2), parallel=True)
c.add_noise(Measure(), PauliX(0.1), parallel=True, before=True)

print(c)

5-qubit circuit with 30 instructions:
├── Reset @ q[0]
├── Reset @ q[1]
├── Reset @ q[2]
├── Reset @ q[3]
├── Reset @ q[4]
├── PauliX(0.1) @ q[0]
├── PauliX(0.1) @ q[1]
├── PauliX(0.1) @ q[2]
├── PauliX(0.1) @ q[3]
├── PauliX(0.1) @ q[4]
├── H @ q[0]
├── AmplitudeDamping(0.1) @ q[0]
├── CX @ q[0], q[1]
├── Depolarizing(0.2) @ q[0,1]
├── CX @ q[0], q[2]
├── Depolarizing(0.2) @ q[0,2]
├── CX @ q[0], q[3]
├── Depolarizing(0.2) @ q[0,3]
├── CX @ q[0], q[4]
├── Depolarizing(0.2) @ q[0,4]
├── PauliX(0.1) @ q[0]
├── PauliX(0.1) @ q[0]
├── PauliX(0.1) @ q[0]
├── PauliX(0.1) @ q[0]
├── PauliX(0.1) @ q[0]
├── M @ q[0], c[0]
├── M @ q[1], c[1]
├── M @ q[2], c[2]
├── M @ q[3], c[3]
└── M @ q[4], c[4]


# Running Noisy Simulations

In [ ]:
# Build a GHZ state with noisy measurements
c = Circuit()
c.push(GateH(), 0)
c.push(GateCX(), 0, range(1,5))
c.push(Measure(), range(5), range(5))

# Add some noise
c.add_noise(Measure(), PauliX(0.1), before=True, parallel=True)

# execute the quantum circuit
quantanium().execute(c)

QCSResults:
├── simulator: QLEO 0.1.22
├── timings:
│    └── apply time: 0.066502476s
├── fidelity estimate:
│    ├── min, max: 1, 1
│    ├── mean: 1
│    ├── median: 1
│    └── std: 0
├── average multi-qubit gate error estimate:
│    ├── min, max: 0, 0
│    ├── mean: 0
│    ├── median: 0
│    └── std: 0
├── most sampled:
│    ├── bs"11111" => 342
│    ├── bs"00000" => 338
│    ├── bs"01111" => 180
│    └── bs"10000" => 140
├── 1000 executions
├── 0 amplitudes
└── 1000 samples

# Sampling Mixed Unitary Channels

In [13]:
import random

# create a new prng with a fixed seed
rng = random.Random(42)

# Create a circuit with mixed unitary noise
c = Circuit()
c.push(Depolarizing1(0.5), [1,2,3,4,5])

# Sample the mixed_unitary noise
c.sample_mixedunitaries(rng=rng, ids=True)

6-qubit circuit with 5 instructions:
├── X @ q[1]
├── I @ q[2]
├── I @ q[3]
├── I @ q[4]
└── Y @ q[5]

# Noise Channels

## Depolarizing Noise

In [14]:
c = Circuit()

c.push(Depolarizing(1, 0.1), 0)
c.push(Depolarizing1(0.2), 0)
c.push(Depolarizing(2, 0.3), 0, 1)
c.push(Depolarizing2(0.5), 0, 1)
c.push(Depolarizing(5, 0.6), *range(5))

5-qubit circuit with 5 instructions:
├── Depolarizing(0.1) @ q[0]
├── Depolarizing(0.2) @ q[0]
├── Depolarizing(0.3) @ q[0,1]
├── Depolarizing(0.5) @ q[0,1]
└── Depolarizing(0.6) @ q[0,1,2,3,4]

In [15]:
Depolarizing(1, 0.1).krausoperators()

[1-qubit Operator:
 ├── 0.948683298050514 0
 └── 0 0.948683298050514,
 1-qubit Operator:
 ├── 0 0.182574185835055
 └── 0.182574185835055 0,
 1-qubit Operator:
 ├── 0 -0.0 - 0.182574185835055*I
 └── 0.0 + 0.182574185835055*I 0,
 1-qubit Operator:
 ├── 0.182574185835055 0
 └── 0 -0.182574185835055]

## Pauli Noise

In [16]:
c = Circuit()

# Single-qubit pauli noise
ps = [0.8, 0.1, 0.1]
paulis = ["I", "X", "Y"]
c.push(PauliNoise(ps, paulis), 0)

# Multi=qubit Pauli noise
ps = [0.9, 0.1]
paulis = ["XY", "II"]
c.push(PauliNoise(ps, paulis), 0, 1)

2-qubit circuit with 2 instructions:
├── PauliNoise((0.8, pauli"I"), (0.1, pauli"X"), (0.1, pauli"Y")) @ q[0]
└── PauliNoise((0.9, pauli"XY"), (0.1, pauli"II")) @ q[0]

## Amplitude Damping

In [17]:
c = Circuit()

c.push(AmplitudeDamping(0.1), 0)

# Show the operators making up the channel
AmplitudeDamping(0.1).krausoperators()

[D(1, np.float64(0.9486832980505138)), SigmaMinus(0.31622776601683794)]

## Thermal Noise

In [18]:
c = Circuit()

tn = ThermalNoise(0.5, 0.6, 1.2, 0.3)
c.push(tn, 1)

2-qubit circuit with 1 instructions:
└── ThermalNoise(0.5, 0.6, 1.2, 0.3) @ q[1]

In [19]:
tn.krausoperators()

[D(np.float64(0.22446947713592924), np.float64(0.6029117408896503)),
 P₀(0.8226960798625129),
 SigmaMinus(0.7978078921002294),
 SigmaPlus(0.522287865083209)]

## Projective Noise

In [20]:
c = Circuit()

c.push(ProjectiveNoise("Z"), 1)
c.push(ProjectiveNoiseX(), 1)

2-qubit circuit with 2 instructions:
├── ProjectiveNoiseZ @ q[1]
└── ProjectiveNoiseX @ q[1]

In [21]:
ProjectiveNoiseX().krausoperators()

[PX₀(1), PX₁(1)]

## Reset

In [22]:
c = Circuit()

c.push(Reset(), range(4))

4-qubit circuit with 4 instructions:
├── Reset @ q[0]
├── Reset @ q[1]
├── Reset @ q[2]
└── Reset @ q[3]

In [23]:
Reset().krausoperators()

[P₀(1.0), SigmaMinus(1.0)]

# Amplitude

In [24]:
# Mistery circuit
c = Circuit()
c.push(GateH(), range(3))

# ask for the amplitude
amp = Amplitude(BitString("101"))
c.push(amp, 0)

3-qubit circuit with 4 instructions:
├── H @ q[0]
├── H @ q[1]
├── H @ q[2]
└── Amplitude(bs"101") @ z[0]

In [ ]:
quantanium().execute(c)

QCSResults:
├── simulator: QLEO 0.1.22
├── timings:
│    ├── sample time: 0.00014129s
│    └── apply time: 3.8553e-05s
├── fidelity estimate: 1
├── average multi-qubit gate error estimate: 0
├── most sampled:
│    ├── bs"011" => 140
│    ├── bs"101" => 133
│    ├── bs"001" => 130
│    ├── bs"110" => 130
│    └── bs"111" => 123
├── zreg (most sampled):
│    └── [(0.3535533905932737+0j)] => 1
├── 1 executions
├── 0 amplitudes
└── 1000 samples

# Expectation Values

In [26]:
# Prepare one qubit in |+>
c = Circuit()
c.push(GateH(), 0)

# Measure the expval of sigma+
op = SigmaPlus()
ev = ExpectationValue(op)
c.push(ev, 0, 0)

1-qubit circuit with 2 instructions:
├── H @ q[0]
└── ⟨SigmaPlus(1)⟩ @ q[0], z[0]

In [ ]:
quantanium().execute(c)

QCSResults:
├── simulator: QLEO 0.1.22
├── timings:
│    ├── sample time: 0.000111049s
│    └── apply time: 6.3905e-05s
├── fidelity estimate: 1
├── average multi-qubit gate error estimate: 0
├── most sampled:
│    ├── bs"0" => 510
│    └── bs"1" => 490
├── zreg (most sampled):
│    └── [(0.4999999999999999+0j)] => 1
├── 1 executions
├── 0 amplitudes
└── 1000 samples